In [1]:
import datetime
from Crypto.Cipher import AES
from Crypto.Hash import MD5, SHA256, HMAC
from Crypto.Util.strxor import strxor

In [2]:
initial_state = bytes.fromhex("b5562ff25e66e602eae4dbd61b2d5e8b")

In [3]:
def prng_generate(state, timestamp_bytes):
    h = MD5.new()
    h.update(strxor(state, timestamp_bytes))
    output = h.digest()
    new_state = strxor(state, output)
    return output, new_state

In [4]:
def generate_candidate_timestamps(base_time, window_minutes=10):
    """Generates candidate timestamps around the base time."""
    candidates = []
    base_dt = datetime.datetime.strptime(base_time, "%Y%m%d%H%M%S")  # Parse base time
    
    # Generate timestamps for a window of `window_minutes` before and after base time
    for delta in range(-window_minutes * 60,0):  # 5-minute window
        candidate_dt = base_dt + datetime.timedelta(seconds=delta)
        for centisecond in range(0, 100):  # Centisecond precision
            timestamp_str = candidate_dt.strftime("%Y%m%d%H%M%S") + f"{centisecond:02d}"
            candidates.append(timestamp_str.encode('ascii'))  # Store as ASCII encoded byte string
    
    return candidates

In [5]:
# Step 2: Generate candidate timestamps
base_time = "20220319121000"  # March 19, 2022, 12:10:00 PM
candidate_timestamps = generate_candidate_timestamps(base_time)


In [6]:
def parse_message(file_path):
    """Parse the message from the binary file."""
    with open(file_path, 'rb') as f:
        msg = f.read()  # Read the entire file content
    
    # Header: first 9 bytes
    header = msg[:9]
    # IV: next 16 bytes
    iv = msg[9:25]
    # MAC: last 32 bytes
    mac = msg[-32:]
    # Encrypted payload: everything in between
    encrypted_payload = msg[25:-32]
    
    return header, iv, encrypted_payload, mac

In [ ]:
# Step 3: Parse the message



In [16]:
def verify_mac(mackey, header, iv, encrypted_payload, mac):
    """Verify the MAC using HMAC and SHA256."""
    h = HMAC.new(mackey, digestmod=SHA256)
    h.update(header)
    h.update(iv)
    h.update(encrypted_payload)
    try:
        h.verify(mac)  # This will raise an exception if the MAC does not match
        return True, mackey.hex()
    except ValueError:
        return False, "mackey NOT found"

In [ ]:
message_file = "message.bin"
header, iv, encrypted_payload, mac = parse_message(message_file)
for candidate in candidate_timestamps:
    mackey, new_state = prng_generate(initial_state, candidate)
    mac_verified, mackey_val = verify_mac(mackey, header, iv, encrypted_payload, mac)
    if mac_verified:
        mac_timestamp_str = candidate.decode()
        break

In [34]:

print(mac_timestamp_str.decode())
mac_timestamp_bytes = mac_timestamp_str.hex().encode('ascii')


2022031912092375


In [ ]:


# Encrypted message components (assumed already extracted)
with open("message.bin", "rb") as f:
    msg = f.read()
header = msg[:9]
iv = msg[9:25]
mac = msg[-32:]
encrypted_payload = msg[25:-32]

def prng_generate(state, timestamp_bytes):
    h = MD5.new()
    h.update(strxor(state, timestamp_bytes))
    output = h.digest()
    new_state = strxor(state, output)
    return output, new_state

def decrypt_payload(enckey, iv, encrypted_payload):
    cipher = AES.new(enckey, AES.MODE_CBC, iv)
    decrypted = cipher.decrypt(encrypted_payload)
    if b'\x80' in decrypted:
        decrypted = decrypted.rstrip(b'\x00')
        if decrypted.endswith(b'\x80'):
            decrypted = decrypted[:-1]
    return decrypted

# Step 1: Generate MAC key and update state
mackey, state = prng_generate(initial_state, mac_timestamp_bytes)

# Step 2: Iterate candidate timestamps for encryption key
start_dt = datetime.datetime.strptime("20220319120923", "%Y%m%d%H%M%S")
for sec_offset in range(0, 38):  # Up to 20220319121000
    current_time = start_dt + datetime.timedelta(seconds=sec_offset)
    for cc in range(0, 100):
        timestamp_str = current_time.strftime("%Y%m%d%H%M%S") + f"{cc:02d}"
        if timestamp_str <= mac_timestamp_str:
            continue  # skip earlier timestamps
        enc_timestamp_bytes = timestamp_str.encode('ascii')
        enckey, _ = prng_generate(state, enc_timestamp_bytes)
        decrypted = decrypt_payload(enckey, iv, encrypted_payload)
        if b"FLAG" in decrypted:
            print(enckey.hex())
            print(f"[+] Found FLAG using timestamp: {timestamp_str}")
            print(decrypted.decode(errors='ignore'))
            break


4acebaa9b1137b59c238ba90c375932c
[+] Found FLAG using timestamp: 2022031912092412
All computers are hackable. This has as much to do with the computer market as it does with the technologies. We prefer our software full of features and inexpensive, at the expense of security and reliability. That your computer can affect the security of Twitter is a market failure. The industry is filled with market failures that, until now, have been largely ignorable. As computers continue to permeate our homes, cars, businesses, these market failures will no longer be tolerable. Our only solution will be regulation, and that regulation will be foisted on us by a government desperate to "do something" in the face of disaster.

FLAG: 76bb5fe76d87d3bff541f78d6c6156b5

Your modern refrigerator is a computer that keeps things cold. Your oven, similarly, is a computer that makes things hot. An ATM is a computer with money inside. Your car is no longer a mechanical device with some computers inside; it's a